In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5105)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [2]:
%matplotlib inline
from __future__ import print_function, division
from IPython.display import FileLink

In [3]:
path="/home/asaeed9/work/data/2cat/sample"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data

In [4]:
batch_size=25

In [5]:
from shutil import copyfile
import os, sys
from glob import glob
import numpy as np
from math import *
from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import image, sequence

Using Theano backend.


In [6]:
%cd /home/asaeed9/work/data/2cat/train

/home/asaeed9/work/data/2cat/train


In [ ]:
# g = glob('*.jpg')
# shuf = np.random.permutation(g)
# for i in range(5000): os.rename(shuf[i], '../valid/' + shuf[i]) 

In [ ]:
# g = glob('*.jpg')
# shuf = np.random.permutation(g)
# for i in range(1000): os.rename(shuf[i], '../test/' + shuf[i]) 

In [8]:
#copy training images
def copy_samples(n):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(n): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(3000): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd /home/asaeed9/work/data/2cat/train

In [9]:
def clean_up():
    %rm $path/train/cats/*
    %rm $path/train/dogs/*
    %rm $path/valid/cats/*
    %rm $path/valid/dogs/*

In [10]:
def get_expected_labels(filenames):
    expected_labels = []
    for elem in filenames:
        if 'dog' in elem:
            expected_labels.append(1)
        elif 'cat' in elem:
            expected_labels.append(0)
    return np.array(expected_labels)

In [11]:
%cd /home/asaeed9/work/data/2cat/train

/home/asaeed9/work/data/2cat/train


In [12]:
clean_up()

In [13]:
def conv_nn(batches, val_batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    model.optimizer.lr = 0.0001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [14]:
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam

In [15]:
%cd /home/asaeed9/work/data/2cat/train
#for n in range(1000, 5001, 1000):
n = 5000
print('sample size: {}'.format(n))
copy_samples(n)

%ls -ltra $path/train/cats/ | wc -l
%ls -ltra $path/train/dogs/ | wc -l
%ls -ltra $path/valid/cats/ | wc -l
%ls -ltra $path/valid/dogs/ | wc -l

gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
            shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

tr_batches = gen_t.flow_from_directory(path + '/train', batch_size=batch_size)
val_batches = gen_t.flow_from_directory(path + '/valid', class_mode='categorical', shuffle=True, batch_size=batch_size)

mod = conv_nn(tr_batches, val_batches)

# gen_test = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
#         shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
test_batches = get_t.flow_from_directory(test_path, batch_size=batch_size)

preds = mod.predict_generator(test_batches, test_batches.nb_sample)


expected_labels = get_expected_labels(test_batches.filenames)
my_labels = np.round(iscat)

print(preds[:5])
print(my_labels[:10])

#latest_weights_filename = 'ft%d.h5' % n
#mod.save_weights(results_path+ '/' +latest_weights_filename)

print ("Completed %s fit operation" % n)
clean_up()

/home/asaeed9/work/data/2cat/train
sample size: 5000
/home/asaeed9/work/data/2cat/valid
/home/asaeed9/work/data/2cat/train
2486
2520
1520
1486
Found 5000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.
Epoch 1/2
5000/5000 [==============================] - 95s - loss: 0.8461 - acc: 0.6252 - val_loss: 1.0548 - val_acc: 0.5210
Epoch 2/2
2950/5000 [================>.............] - ETA: 22s - loss: 0.6315 - acc: 0.6756

MemoryError: Error allocating 206451200 bytes of device memory (out of memory).
Apply node that caused the error: GpuElemwise{Composite{(i0 * (i1 + Abs(i1)))},no_inplace}(CudaNdarrayConstant{[[[[ 0.5]]]]}, GpuElemwise{Add}[(0, 0)].0)
Toposort index: 161
Inputs types: [CudaNdarrayType(float32, (True, True, True, True)), CudaNdarrayType(float32, 4D)]
Inputs shapes: [(1, 1, 1, 1), (25, 32, 254, 254)]
Inputs strides: [(0, 0, 0, 0), (2064512, 64516, 254, 1)]
Inputs values: [CudaNdarray([[[[ 0.5]]]]), 'not shown']
Outputs clients: [[GpuDnnBatchNormGrad{mode='spatial', epsilon=0.0010000000474974513}(GpuElemwise{Composite{(i0 * (i1 + Abs(i1)))},no_inplace}.0, if{inplace,gpu}.0, GpuDimShuffle{x,0,x,x}.0, GpuDnnBatchNorm{mode='spatial', epsilon=0.0010000000474974513, running_average_factor=0.10000000149011612, running_averages=False, inplace_running_mean=False, inplace_running_var=False, inplace_output=False}.1, GpuDnnBatchNorm{mode='spatial', epsilon=0.0010000000474974513, running_average_facto

In [ ]:

iscat = preds[:, 0]
print(preds[:10])
filenames = test_batches.filenames
print(filenames[:10])

### Load weight files and run on test data 

In [ ]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import accuracy_score, log_loss

onlyfiles = [f for f in listdir(path+'/results/') if isfile(join(path+'/results/', f))]

x=[]
y_accuracy=[]
y_logloss = []
gen_test = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
        shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
test_batches = gen_test.flow_from_directory(test_path, batch_size=32)

for f in onlyfiles:
    mod.load_weights(results_path+'/' + f)
    preds = mod.predict_generator(test_batches, test_batches.nb_sample)

    filenames = test_batches.filenames

    print(filenames[:5])

    iscat = preds[:, 0]
    my_labels = np.round(iscat)

    expected_labels = get_expected_labels(test_batches.filenames)

    #     cm = confusion_matrix(expected_labels, my_labels)
    #     plot_confusion_matrix(cm, test_batches.class_indices)
    k = int(f[2:-3])
    #plot k vs training size
    #print(expected_labels)
    #print(my_labels)

    print(accuracy_score(expected_labels, my_labels))
    x.append(k)
    y_accuracy.append(accuracy_score(expected_labels, my_labels))
    y_logloss.append(log_loss(expected_labels, preds))

y_accuracy
y_logloss

In [ ]:

print(test_batches.filenames[:5])

expected_labels = get_expected_labels(test_batches.filenames)

print(expected_labels)

#     cm = confusion_matrix(expected_labels, my_labels)
#     plot_confusion_matrix(cm, test_batches.class_indices)
# k = int(f[2:-3])
# #plot k vs training size
# print(expected_labels)
# print(my_labels)
